# Шаги по предобработки данных, полученных из разных источников

Сложность данного этапа заключается в том, что присутствуют разные провайдеры данных, с различной структурой данных.

В данном ноутбуке будут представлены основные шаги по обработке данных, полученных из каждого источника.

## Обработка данных по оптовому рынку электроэнергетики.

1. Скачиваем данные по адресу https://br.so-ups.ru/BR/GenConsum
2. Данные содержат почасовые прогнозные и фактические данные, относящиеся региону ОЭС Средней Волги. Сворачиваем фактическое потребление до посуточных значений.
3. Сохраняем в формате csv для дальшейшей обработки.

## Обработка данных Сбериндекса

1. Скачиваем данные по адресу:
 - https://sberindex.ru/ru/dashboards/indeks-potrebitelskoi-aktivnosti
 - https://sberindex.ru/ru/dashboards/izmenenie-aktivnosti-msp-po-regionam
2. Оставляем данные, относящиеся к интересующему нас региону

Сниппет для отбора регионов:

In [ ]:
area = ['Пензенская область', 'Самарская область', 'Саратовская область',
    'Ульяновская область', 'Нижегородская область', 'Чувашская Республика',
    'Республика Марий Эл', 'Мордовия', 'Республика Татарстан',]
df = df[df["Регион"].isin(area)]

3. Сохраняем в формате csv для дальшейшей обработки.

## Обработка индекса самоизоляции от Yandex

1. Скачиваем данные по адресу:
 - https://datalens.yandex/7o7is1q6ikh23?tab=q6
2. Используя сервис **dadata.ru** отбираем города, относящиеся к данной Объединенной энергетической системе.
3. Сохраняем в формате csv для дальшейшей обработки.

## Обработка данных по температуре

1. Скачать данные за требуемые периоды с сайта: ftp://ftp.ncdc.noaa.gov/pub/data/gsod/
2. Распаковать данные следующим сниппетом:

In [ ]:
def data_year(year):
    df = pd.DataFrame(columns=["STN---", "YEARMODA", "TEMP", "VISIB", "REGION"])
    with tarfile.open(f"../gsod_{year}.tar", "r:*") as tar:
        for tar_name in tar.getnames():
            if len(tar_name) > 2 and int(tar_name.replace("./", "").split("-")[0]) in stations:
                data_year = pd.read_fwf(tar.extractfile(tar_name), 
                                        widths=[6, 8, 8, 8, 3, 9, 3, 8, 3, 8, 2, 8], 
                                        compression='gzip')[["STN---", "YEARMODA", "TEMP", "VISIB"]].dropna()
                data_year["YEAR"] = data_year["YEARMODA"] // 10000
                data_year["MONTH"] = (data_year["YEARMODA"] - data_year["YEAR"] * 10000) // 100
                data_year["DAY"] = data_year["YEARMODA"] - data_year["YEAR"] * 10000 - data_year["MONTH"] * 100
                data_year["REGION"] = data_year["STN---"].map(stations)
                df = df.append(data_year)
    print(year)
    return df

3. Используя следующий сниппет отбираем метеостанции относящиеся к конкретным регионам.

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

for row, item in df.iterrows():
    lat = item["LAT"]
    lon = item["LON"]
    if not pd.isna(lat) and not pd.isna(lon) and pd.isna(item["region_with_type"]):
        overpass_q = f"""
[out:json][timeout:300];
is_in({lat}, {lon})->.areas;
rel(pivot.areas)[boundary=administrative][admin_level~"^[4]$"];
out geom;
"""
        response = requests.get(overpass_url, params={'data': overpass_q})
        if "rate_limited" in response.text or "runtime error" in response.text:
            print("Timeout", lat, lon)
            time.sleep(300)
        else:
            time.sleep(3)
            if len(response.json()["elements"]):
                tags = response.json()["elements"][0]["tags"]
                if "ref" in tags:
                    df.loc[row, 'region_with_type'] = tags["name"]
                    df.loc[row, 'region_iso_code'] = tags["ref"]
                    print(row, lat, lon, tags["name"], tags["ref"])

4. Отобрать данные только по требуемому ОЭС
5. Сохраняем в формате csv для дальшейшей обработки

Для каждого этапа получения и обработки данных возможна полная автоматизация процесса, предусмотренная соответствующим лицензионным соглашением.